In this notebook, I create the citation vector dataframe for each year for section H

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import pickle

from collections import Counter

In [2]:
data_drct = '/home/somayeh/Documents/Career/Data_Incubator/Capstone/Data/'
result_drct = '/home/somayeh/Documents/Career/Data_Incubator/Capstone/Result/'
filename1 = 'application.tsv'
filename2 = 'cpc_current.tsv'
filename3 = 'uspatentcitation.tsv'

patent issuance year is obtained in "Patent_query-Oct10-Copy1.ipynb, and stored in:

/home/somayeh/Documents/Career/Data_Incubator/Project/patent_yr

Here, I need the patent_id, citation_id, and year. For that, I am creating another (previous one is redundant) set of yearly dataframes containing all those information (as below):

In [ ]:
df_chunk = pd.read_csv(data_drct+filename2,sep='\t',header=0, chunksize=1000000)
patent_list2 = []  
for cnt, chunk in enumerate(df_chunk):
#     print(cnt)    
    chunk = chunk[chunk["section_id"] == "H"]
    chunk = chunk[["patent_id","group_id","category","sequence"]]
    chunk["category"] = pd.Categorical(chunk["category"])
    chunk["patent_id"] = chunk["patent_id"].astype(int)
    patent_list2.append(chunk)
    
patent_id_group = pd.concat(patent_list2)
del patent_list2

In [ ]:
patent_id_group.head()

In [ ]:
len(patent_id_group)

In [ ]:
patent_id_group.to_pickle(result_drct+"/patent_id_group.pkl")

In [ ]:
del patent_id_group

List all patents available after 1960 and their issuance year (to be used later here)

In [ ]:
def get_patent_year(yr):
    drct = '/home/somayeh/Documents/Career/Data_Incubator/Project/'
    fname = drct+'/patent_yr/'+str(yr)+'.pkl'
    with open(fname, 'rb') as f:
        df = pickle.load(f)
    return(df)
     
# get list of patent ids stored in patent_id_group data file   
fname = result_drct+"/patent_id_group.pkl"
with open(fname, 'rb') as f:
    df = pickle.load(f)
patent_list1 = df.patent_id.unique()
del df

# filter citation ids and their year
df_1 = pd.DataFrame()
for yr in range(1960,2018):
    print(yr)
    df1 = get_patent_year(yr)   
    df1 = df1[df1.citation_id.str.isnumeric()]
    df1.citation_id = df1.citation_id.astype(int)
    df1[df1.citation_id.isin(patent_list1)]
    df_1 = pd.concat([df_1,df1])

In [ ]:
df_1.to_pickle(result_drct+"/patent_id_year.pkl")

In [ ]:
df_1.head()

In [ ]:
len(df_1.citation_id.unique()), len(df_1.citation_id)

In [ ]:
del df_1

__Reading patent groups from file:__

In [ ]:
fname = result_drct+"/patent_id_group.pkl"
with open(fname, 'rb') as f:
    patent_id_group = pickle.load(f)
# patent_id_group = patent_id_group.drop_duplicates() 
patent_id_group = patent_id_group.sort_values(by=['patent_id', 'group_id'])
patent_id_group = patent_id_group.drop(columns = ["category","sequence"], axis=1)
patent_id_group = patent_id_group.reset_index(drop=True)
patent_id_group.head()

In [ ]:
len(patent_id_group.patent_id.unique()), len(patent_id_group.patent_id)

Reading patent ids and issuance year:

In [3]:
def get_df_per_year(yr):
    
    drct = '/home/somayeh/Documents/Career/Data_Incubator/Project/'
    fname = drct+'/patent_citation_yr/'+str(yr)+'.pkl'
    with open(fname, 'rb') as f:
        df = pickle.load(f)
    df = (df
            .drop(['patent_id','citation_id', 'year'], axis=1)
            .join(df[['patent_id','citation_id', 'year']].apply(pd.to_numeric, errors='coerce')))
    df = df.dropna()
    df[['patent_id','citation_id']] = df[['patent_id','citation_id']].astype(int)
    df = df.reset_index(drop=True)
    
    result_drct = '/home/somayeh/Documents/Career/Data_Incubator/Capstone/Result/'
    fname = result_drct+"/patent_id_group.pkl"
    with open(fname, 'rb') as f:
        patent_id_group = pickle.load(f)
    patent_id_group = patent_id_group[["patent_id", "group_id"]]
    patent_id_group = patent_id_group.loc[patent_id_group["patent_id"].isin(df.patent_id)]

    df = pd.merge(df, patent_id_group, how='left', on=['patent_id'])

    df = df.rename(columns={'year': 'year_citation', 'group_id':'patent_group','patent_id': 'p_id', 'citation_id':'patent_id'})
    df = pd.merge(df, patent_id_group, how='left', on=['patent_id'])
    df = df.rename(columns={'p_id': 'patent_id', 'patent_id':'citation_id', 'group_id':'citation_group'})
    df = df.dropna()
    df = df.reset_index(drop=True)
    del patent_id_group 

    result_drct = '/home/somayeh/Documents/Career/Data_Incubator/Capstone/Result/'
    fname = result_drct+"/patent_id_year.pkl"
    with open(fname, 'rb') as f:
        df2 = pickle.load(f)
    df2 = df2.loc[df2["year"]>=yr-3]
    df2["year"] = df2["year"].astype(int)
    df2 = df2.rename(columns={'year': 'year_patent', 'citation_id':'patent_id'})
    df = pd.merge(df, df2, how='left', on=['patent_id'])
    df = df.dropna()
    
    
    return(df)

In [15]:
df = get_df_per_year(2012)
df = df.drop_duplicates()

all_links = list(zip(df.patent_group, df.citation_group))
unique_links = set(list(zip(df.patent_group, df.citation_group)))

dd = Counter(all_links)
mc = dd.most_common(10)
mc

[(('H04L', 'H04L'), 2918),
 (('H04W', 'H04W'), 1409),
 (('H04W', 'H04L'), 1389),
 (('H04L', 'H04W'), 1286),
 (('H01L', 'H01L'), 889),
 (('H04N', 'H04N'), 855),
 (('H04L', 'H04M'), 635),
 (('H04M', 'H04L'), 632),
 (('H04W', 'H04M'), 601),
 (('H04M', 'H04W'), 539)]

In [11]:
#List of all unique groups in H section
# get list of group ids stored in patent_id_group data file   
drct = '/home/somayeh/Documents/Career/Data_Incubator/Project/'
fname = result_drct+"/patent_id_group.pkl"
with open(fname, 'rb') as f:
    df2 = pickle.load(f)
group_list = df2.group_id.unique()
del df2

In [14]:
group_list

array(['H01J', 'H01L', 'H05F', 'H01H', 'H02J', 'H02P', 'H01S', 'H01C',
       'H03K', 'H03C', 'H03G', 'H02N', 'H02K', 'H04B', 'H03H', 'H04L',
       'H01F', 'H04W', 'H03M', 'H01Q', 'H02S', 'H03J', 'H03L', 'H03F',
       'H05K', 'H01P', 'H04N', 'H04J', 'H01T', 'H01B', 'H01R', 'H01M',
       'H05C', 'H02H', 'H05B', 'H01G', 'H02M', 'H04R', 'H05H', 'H02G',
       'H02B', 'H04H', 'H04M', 'H04Q', 'H01K', 'H03B', 'H03D', 'H04S',
       'H05G', 'H04K'], dtype=object)

In [7]:
len(all_links), len(unique_links), len(df.patent_group.unique())

(22287, 567, 48)

In [19]:
node_names, adj_idx = np.unique(all_links, return_inverse=True)

In [22]:
node_names, adj_idx

(array(['H01B', 'H01C', 'H01F', 'H01G', 'H01H', 'H01J', 'H01L', 'H01M',
        'H01P', 'H01Q', 'H01R', 'H01S', 'H01T', 'H02B', 'H02G', 'H02H',
        'H02J', 'H02K', 'H02M', 'H02N', 'H02P', 'H02S', 'H03B', 'H03C',
        'H03D', 'H03F', 'H03G', 'H03H', 'H03J', 'H03K', 'H03L', 'H03M',
        'H04B', 'H04H', 'H04J', 'H04K', 'H04L', 'H04M', 'H04N', 'H04Q',
        'H04R', 'H04S', 'H04W', 'H05B', 'H05F', 'H05G', 'H05H', 'H05K'],
       dtype='<U4'), array([38, 38,  6, ..., 21, 16, 21]))

In [26]:
adj_idx = adj_idx.reshape(-1, 2)
adj_matrix = np.zeros((len(node_names),)*2)
adj_matrix[adj_idx[:, 0], adj_idx[:, 1]] = 1
adj_matrix[1]

array([0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.])

In [ ]:
df = get_df_per_year(1980)

all_links = list(zip(df.patent_group, df.citation_group))
unique_links = set(list(zip(df.patent_group, df.citation_group)))

dd = Counter(all_links)
mc = dd.most_common(50)
mc

In [ ]:
nodes = df.patent_group.unique()
All_Links = list(zip(df.patent_group, df.citation_group))

G = nx.MultiGraph()
G.add_edges_from(All_Links)
M = nx.to_numpy_matrix(G)

In [ ]:
from numpy import linalg as LA
val, vec = LA.eig(M)

In [ ]:
max(val)

In [ ]:
ind = np.argmax(val)
max(np.array(vec)[ind])

In [ ]:
df = get_df_per_year(2000)

all_links = list(zip(df.patent_group, df.citation_group))
unique_links = set(list(zip(df.patent_group, df.citation_group)))

dd = Counter(all_links)
mc = dd.most_common(100)
mc

Create network of citation for groups:

In [ ]:
df = get_df_per_year(2000)

In [ ]:
def draw_graph(G):
    pos=nx.spring_layout(G) # positions for all nodes
    # nodes
    nx.draw_networkx_nodes(G,pos, node_color='r',node_size=5,alpha=0.4)
    #edges
    nx.draw_networkx_edges(G,pos,width=1.0,alpha=0.9)

In [ ]:
import networkx as nx

nodes = df.patent_group.unique()
Links = set(list(zip(df.patent_group, df.citation_group)))

G = nx.Graph()  # initiate a graph
G.add_nodes_from(list(nodes)) # add nodes to the graph. Here, the nodes are the unique authors
G.add_edges_from(Links) # adding edge based on coauthorship


remove = [node for node,degree in dict(G.degree()).items() if degree < 200]
G.remove_nodes_from(remove)

draw_graph(G)
print(nx.info(G))

In [ ]:
sub_graphs = nx.connected_component_subgraphs(G)
len(sub_graphs)


In [ ]:
# del G 
nodes = df.patent_group.unique()
Links = list(zip(df.patent_group, df.citation_group))

G2 = nx.MultiGraph()
G2.add_nodes_from(list(nodes)) # add nodes to the graph. Here, the nodes are the unique authors
G2.add_edges_from(Links) # adding edge based on coauthorship


# remove = [node for node,degree in dict(G2.degree()).items() if degree < 50]
# G2.remove_nodes_from(remove)

# draw_graph(G)
print(nx.info(G2))

In [ ]:
len(Links)

In [ ]:
patent_citation_id_year = (patent_citation_id_year
            .drop(['patent_id','citation_id', 'year'], axis=1)
            .join(patent_citation_id_year[['patent_id','citation_id', 'year']].apply(pd.to_numeric, errors='coerce')))
patent_citation_id_year[['patent_id','citation_id']] = patent_citation_id_year[['patent_id','citation_id']].fillna(0).astype(int)

In [ ]:
patent_citation_id_year.head()

In [ ]:
len(patent_id_group), len(patent_id_group.patent_id.unique())

In [ ]:
# p_group = []
# for i in range(10):
#     patent_id,citation_id, year = patent_citation_id_year.iloc[i]
# #     print(i)
#     if i>10:
#         break
#     p_group.append( patent_id_group.loc[patent_id_group.patent_id ==patent_id])
# # print(patent_id, p_group)

In [ ]:
# result = patent_citation_id_year.join(patent_id_group, on=['patent_id'], how='inner')
patent_citation_id_year2 = pd.merge(patent_citation_id_year, patent_id_group, how='right', on=['patent_id'])


In [ ]:
len(patent_citation_id_year2)

In [ ]:
result = (result
            .drop(['patent_id','citation_id', 'year'], axis=1)
            .join(result[['patent_id','citation_id', 'year']].apply(pd.to_numeric, errors='coerce')))
result[['patent_id','citation_id', 'year']] = result[['patent_id','citation_id', 'year']].fillna(0).astype(int)

In [ ]:
len(result.group_id.unique())
len(result.citation_id.unique())

In [ ]:
patent_citation_id_year.head()

In [ ]:
df_tmp = patent_citation_id_year.loc[patent_citation_id_year.patent_id.isin(patent_id_group.patent_id.astype(str))]

In [ ]:
df_tmp2= patent_citation_id_year.loc[patent_citation_id_year.citation_id.isin(patent_id_group.patent_id.astype(str))]

In [ ]:
len(patent_citation_id_year), len(df_tmp),len(df_tmp2)

In [ ]:
len(df_tmp.patent_id.unique())

In [ ]:
df_tmp2 = patent_id_group.loc[patent_citation_id_year.patent_id.isin(patent_id_group.patent_id.astype(str))]

In [ ]:
patent_citation_id_year.patent_id[6]

In [ ]:
patent_citation_id_year.patent_id

In [ ]:
patent_id_year = patent_id_year.drop_duplicates(subset='citation_id', keep = 'last')

In [ ]:
len(patent_id_year), len(patent_id_year.citation_id.unique())

Plotting the number distribution of patents over time:

In [ ]:
count =[]
year = []
for yr in range(1960,2018):
    year.append(yr)
    count.append(len(patent_id_year.loc[patent_id_year.year == yr]))
plt.plot(year,count)
plt.xlabel('Year')
plt.ylabel('Number of Issued Patents')
# plt.legend( )

In [ ]:
# df_chunk = pd.read_csv(drct+filename2,sep='\t',header=0, chunksize=1000000)
# patent_list2 = []  
# for cnt, chunk in enumerate(df_chunk):
#     print(cnt)    
# #     chunk.drop(['patent_id','citation_id','uuid', 'name', 'kind','country','category','sequence'], axis=1)
# #     chunk['year'] = pd.to_datetime(chunk['date'], format = '%Y-%M-%d', errors='coerce').dt.year
#     patent_list2.append(chunk[['patent_id','group_id']])
# #     if cnt>1:
# #         break
    
# patent_id_group = pd.concat(patent_list2)
# del patent_list2  


In [ ]:
# fname = drct+'patent_id_group'
# with open(fname, 'wb') as f:
#     pickle.dump(patent_id_group, f)


In [ ]:
fname = drct+'patent_id_group'
with open(fname, 'rb') as f:
    patent_id_group = pickle.load(f)

In [ ]:
len(patent_id_group.patent_id.unique()), len(patent_id_group)

In [ ]:
patent_id_group = patent_id_group.drop_duplicates()

In [ ]:
len(patent_id_group.drop_duplicates().patent_id.unique()), len(patent_id_group)

In [ ]:
df_tmp = patent_id_year.loc[patent_id_year.citation_id.isin(patent_id_group.patent_id.astype(str))]

In [ ]:
len(df_tmp)

In [ ]:
patent_id_group.group_id.unique()

__NOTE:__
D in front of patent id indicates "design" patent, in contrast to "utility" patent.

In general terms, a “utility patent” protects the way an article is used and works (35 U.S.C. 101), while a “design patent” protects the way an article looks (35 U.S.C. 171). The ornamental appearance for an article includes its shape/configuration or surface ornamentation applied to the article, or both. Both design and utility patents may be obtained on an article if invention resides both in its utility and ornamental appearance.

In [ ]:
# distinct_groups = df.unique()

# fname = drct+'distinct_groups'
# with open(fname, 'wb') as f:
#     pickle.dump(distinct_groups, f)


Strategy: Create a (dataframe) table for each year. 

Each row indicate a patent issued up until now. Followed by the citation vector.
Note that the citation vector is updated for each citation each year, and new rows are added (as new patents are issued)


In [ ]:
# patent_citation_id_year = pd.DataFrame()
# for yr in range(1965,1966):
#     fname = drct+'/patent_citation_yr/'+str(yr)+'.pkl'
#     with open(fname, 'rb') as f:
# #         print(fname)
#         df = pickle.load(f)
#         patent_citation_id_year = pd.concat([patent_citation_id_year,df])
